
micropython clone on a linux / WSL drive 

+ Docker + mpbuild 

check out version 


for platform in ("unix standard", "webassembly standard", "webassembly pyscript", "windows standard")
    `mpbuild clean platform`
    `mpbuild build platform`
    if keep:
        add / commit the firmware 
    run sa_micropython -p creatstubs.py 
    process the output



## Createstubs for Micropython-Windows-port
- firmware for windows can be built on WSL using mpbuild or the `sa_ports_build.py` script
- firmware must be accessible from the windows machine ( copied to the windows drive )
- micropython.exe Needs to be run on windows 


In [28]:
import platform
from pathlib import Path
from stubber.utils.config import CONFIG

ROOT_PATH = CONFIG.config_path
from mpflash.versions import get_preview_mp_version, get_stable_mp_version

In [27]:
from mpflash.downloaded import find_downloaded_firmware
from mpflash.config import config as flash_config


def get_sa_firmware_path(
    board_id: str,  # = "unix-standard",
    version: str = get_stable_mp_version(),
):
    fws = find_downloaded_firmware(
        board_id=board_id,
        custom=True,
        # variants=True,
        version=get_stable_mp_version(),
    )
    for fw in fws:
        fw_path = flash_config.firmware_folder / fw.firmware_file
        if fw_path.exists():
            return fw_path
    return None


get_sa_firmware_path("unix-standard")

2025-12-18 23:24:03.258 | DEBUG    | mpflash.downloaded:find_downloaded_firmware:65 - Looking for firmware for unix-standard v1.27.0 


PosixPath('/home/jos/Downloads/firmware/unix/unix-standard-v1.27.0')

In [30]:
firmware_path  = get_sa_firmware_path("unix-standard").as_posix()
print(f"Using firmware at: {firmware_path}")
! chmod +x {firmware_path}
! ls -l {firmware_path}
print(f"{firmware_path} {createstubs_py} --path {dest}")
# !{firmware_path} {createstubs_py} --path {dest}

2025-12-18 23:24:49.153 | DEBUG    | mpflash.downloaded:find_downloaded_firmware:65 - Looking for firmware for unix-standard v1.27.0 


Using firmware at: /home/jos/Downloads/firmware/unix/unix-standard-v1.27.0


1788.35s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1793.60s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


-rwxr-xr-x 1 jos jos 699896 Dec 18 22:42 /home/jos/Downloads/firmware/unix/unix-standard-v1.27.0
/home/jos/Downloads/firmware/unix/unix-standard-v1.27.0 /home/jos/micropython-stubber/src/stubber/board/createstubs.py --path /home/jos/micropython-stubber/repos/micropython-stubs


In [31]:
# createstubs for the Windows port
# - port,board,variant = windows,None,standard
# todo : add board / port.py
# - stubber merge expects - port or GENERIC - ( todo : Add standard variant to stubber merge )

firmware = str(ROOT_PATH / "firmware")
dest = str(ROOT_PATH / "repos" / "micropython-stubs")
createstubs_py = str(ROOT_PATH / "src" / "stubber" / "board" / "createstubs.py")

port = "unix"
for version in [
    # "v1.25.0",
    # "v1.26.0",
    # "v1.26.1",
    get_stable_mp_version(),
    get_preview_mp_version(),
]:
    firmware_path = get_sa_firmware_path(
        f"{port}-standard",
        version=version,
    )
    # port = platform.system().lower()
    if platform.system() == "Windows" and port:
        # run createstubs.py
        !{firmware_path} {createstubs_py} --path {dest}
        pass
    elif platform.system() == "Linux":
        # make executable and run
        !chmod +x {firmware_path}
        !{firmware_path} {createstubs_py} --path {dest}
    else:
        print("Unsupported platform")

    for port in ["windows", "unix"]:
        # merge and build
        !stubber merge --version {version} --port {port}
        !stubber build --version {version} --port {port}

        # Publish package to pypi
        publish = False
        if publish:
            !stubber publish --version stable --port {port} --pypi

# update release json
# push to github


2025-12-18 23:29:41.295 | DEBUG    | mpflash.downloaded:find_downloaded_firmware:65 - Looking for firmware for unix-standard v1.27.0 
2080.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
2085.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


createstubs.py: v1.26.4
INFO  : Port: unix
INFO  : Board: standard
INFO  : Board_ID: standard
INFO  : Clean/remove files in folder: /home/jos/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_27_0-unix-standard
INFO  : Clean/remove files in folder: /home/jos/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_27_0-unix-standard/mip
INFO  : Clean/remove files in folder: /home/jos/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_27_0-unix-standard/uasyncio
INFO  : Clean/remove files in folder: /home/jos/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_27_0-unix-standard/asyncio
INFO  : Clean/remove files in folder: /home/jos/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_27_0-unix-standard/requests
INFO  : Start micropython-stubber v1.26.4 on micropython-v1.27.0-unix-standard
INFO  : Report file: /home/jos/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_27_0-unix-standard/modules.json
INFO  : S

2091.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


23:29:58 | ℹ️ micropython-stubber 1.26.4
23:29:58 | ℹ️ Merge docstubs for micropython ['v1.27.0']
23:29:58 | ℹ️ checking 1 possible board candidates
23:29:58 | ℹ️ merged 0 of 1 candidates


2098.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

23:30:49 | ℹ️ micropython-stubber 1.26.4
23:30:49 | ℹ️ Build micropython ['v1.28.0-preview'] ['windows'] ['GENERIC']
23:30:49 | ℹ️ checking 1 possible board candidates
23:30:49 | ℹ️ Built 0 stub packages
              Build Results               
┏━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━┓
┃ result ┃ name ┃ version ┃ error ┃ path ┃
┡━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━┩
└────────┴──────┴─────────┴───────┴──────┘


2149.47s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


23:30:56 | ℹ️ micropython-stubber 1.26.4
23:30:56 | ℹ️ Merge docstubs for micropython ['v1.28.0-preview']
23:30:56 | ℹ️ checking 1 possible board candidates
23:30:56 | ℹ️ Merge v1.28.0-preview docstubs with boardstubs to 
micropython-v1_28_0_preview-unix-standard-merged
23:30:56 | ℹ️ Removing modules from the merged stubs
23:30:56 | ℹ️ create umodules to refer to modules in the merged stubs
23:30:56 | ℹ️ Enriching from 
repos/micropython-stubs/stubs/micropython-v1_28_0_preview-docstubs to 
repos/micropython-stubs/stubs/micropython-v1_28_0_preview-unix-standard-merged/*
*/*.py*
23:30:56 | ℹ️ Enriching file: 
repos/micropython-stubs/stubs/micropython-v1_28_0_preview-unix-standard-merged/_
thread.pyi
23:30:56 | ℹ️ Enriching file: 
repos/micropython-stubs/stubs/micropython-v1_28_0_preview-unix-standard-merged/b
inascii.pyi
23:30:56 | ℹ️ Enriching file: 
repos/micropython-stubs/stubs/micropython-v1_28_0_preview-unix-standard-merged/b
tree.pyi
23:30:56 | ℹ️ Enriching file: 
repos/micropython

2168.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


23:31:15 | ℹ️ micropython-stubber 1.26.4
23:31:15 | ℹ️ Build micropython ['v1.28.0-preview'] ['unix'] ['GENERIC']
23:31:15 | ℹ️ checking 1 possible board candidates
23:31:16 | ℹ️ Build: micropython-v1_28_0_preview-unix-stubs
23:31:16 | ℹ️ Found changes to package sources: micropython-unix-stubs 1.28.0a1 
Using virtualenv: /home/jos/micropython-stubber/.venv
23:31:16 | ℹ️ Built 1 stub packages
                                 Build Results                                  
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ result   ┃ name                   ┃ version  ┃ error ┃ path                  ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ Build OK │ micropython-unix-stubs │ 1.28.0a1 │ None  │ repos/micropython-st… │
└──────────┴────────────────────────┴──────────┴───────┴───────────────────────┘


## Createstubs for Micropython-unix port
- Needs to be run on linux / wsl 
- firmware for windows can be built on WSL using mpbuild using the `build_sa_ports.py` script
- make firmware executable
- requires manual rename of the stub-folder to -GENERIC ( FIXME) 
- runn the stubber command from the `micropython-stubs` repo 
- Note that the loaction where `stubber` is run determines where it will look for the stubs 

In [ ]:
firmware = "/home/jos/micropython-stubber/firmware"
dest = "/home/jos/micropython-stubs"
createstubs_py = "/home/jos/micropython-stubber/src/stubber/board/createstubs.py"
port = "unix"
for version in ["v1.24.1", "v1.25.0-preview"]:
    # run createstubs.py
    !chmod +x {firmware}/{port}/{port}-{version}
    !{firmware}/{port}/{port}-{version} {createstubs_py} --path {dest}
    # merge and build
    # !stubber merge --version {version} --port {port}
    # !stubber build --version {version} --port {port}

# Publish package to pypi
publish = False
if publish:
    !stubber publish --version stable --port window --pypi

# update release json
# push to github